<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST3512/blob/main/CST3512_CL05_MapBox_14_Feb_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CST3512 - Class 05**

This notebook demonstrates exercises to produce a map from geo location (longitude and latitude) data using Mapbox and plotly.  That output is then used by **Kaleido** and **plotly Chart-Studio** to be saved as a static image in a variety of formats.    

In addition to plotting geocade data, this exercise will demonstrate the creation of free accounts and initiation of API keys for the following services:
* Mapbox
* plotly Chart-Studio


# MapHub, Crowd-sourced Russian-Ukraine Monitor    

*by Cen4infoRes*    

Before working with **Mapbox**, **plotly** and related libraries, consider this crowdsourced example of dynamic map data.  The following link to an interactive map presented using **MapHub** includes data derived from several different social media sources.

[Russian-Ukraine Monitor Map](https://maphub.net/Cen4infoRes/russian-ukraine-monitor)     


The Russia-Ukraine Monitor Map is a crowdsourced effort to map, document and verify information in order to provide reliable information for policymakers and journalists of the on-the-ground and online situation in and around Ukraine.

This map is created by the [Centre For Information Resilience](https://twitter.com/Cen4infoRes) as part of a wider effort to tackle disinformation and promote transparent and verified information.






---



#Maps in Python with Mapbox, Plotly, Kaleido, and Chart Studio

from: **Working with maps in python using Mapbox and Plotly**
 by: Rahul Bhadani in [Medium](https://medium.com/technology-hits/working-with-maps-in-python-with-mapbox-and-plotly-6f454522ccdd) on 20-Nov-2020.    

Connect with [Rahul Bhadani on LinkedIn](https://www.linkedin.com/in/rahulbhadani/). 


 *The [gps.csv](https://raw.githubusercontent.com/rahulbhadani/medium.com/master/data/gps.csv) data file from the original article is accessed by this notebook.*

There are plenty of mapping services available dominated by Google Map, Here map, OpenStreetMaps, and a few lesser-known as such Mapbox. They vary in features, services, and applicable use cases. For mapping applications that require precision and finer control, Google Map is the choice, but it is not convenient and free for all purposes. Some use-cases of Google MAP API are free but may still require credit card information for billing purposes in order to prevent any misuse.     

A research group started gathering GPS data from consumer cars and was looking to overlay the GPS data on maps for further downstream analysis of GPS data. In the quest for a suitable python API, uses of plotly that employed various types of maps, with or without the need for an API key were considered. plotly provides support of Mapbox, a convenient tool.    
 
Mapbox provides two sets of API token that can be used for querying mapping service: public and private. plotly uses a public API key to query mapping service from Mapbox. However, plotly doesn't necessarily need an API token from Mapbox as it has a support mapping without an API token as well, but that does not allow for the use of Mapbox mapping tiles. Also, API-token-free mapping calls in plotly have poor map rendering. However, API token generation in Mapbox is so easy that there is no reason not to use it. Here is an example.



---

Contents:
1. API Token Generation
2. Library Imports
3. Data File Specification
4. Downloading Plotly Charts as Images in Colab
5. Mapbox and Plotly Figure Generation
6. Kaleido Writing Figure to Files
7. Publishing on Chart Studio


## API Token generation
To generate a Mapbox API token, go to **mapbox.com**, and signup for an account. Fom a mapbox account, generate a token at https://account.mapbox.com/. Press `+create a token` button and it will generate two tokens: **public** and **secret**. Note down the **public** token that you will need for use in plotly.

A map style is needed.  Map styles can be created at https://studio.mapbox.com/. There is an existing map style to use in this exercise --  access URL is `mapbox://styles/strym/ckhe4yk7d04hi19kcu9878xyv`

## Library Imports

Install libraries:
* pandas - for dataframes and analysis
* plotly - for graphs and plots
* chart_studio - for plotly chart studio publishing
* numpy - for calculations
* kaleido - to print figures (graphs and plots)

In [1]:
# import pandas for analysis and dataframes
import pandas as pd

In [15]:
# import numpy for calculations
import numpy as np

In [2]:
# install the latest version of plotly 
!pip install plotly>=4.0.0 
# import requisite plotly methods for charts/plots
import plotly.express as px
import plotly.io as pio
import plotly.offline as pyo
import plotly.graph_objects as go

# Use the following to set notebook mode to work in offline mode
# This is an idempotent method which can and should be called from any
# offline methods that require plotly. js to be loaded into the notebook dom.
# pyo.init_notebook_mode()  # uncomment statement for offline mode

In [ ]:
# install chart_studio to the Colab session
! pip install chart_studio
# import chart_studio
import chart_studio

In [ ]:
# import kaleido to write figure images to files
!pip install -U kaleido

In [ ]:
# Use the following to set notebook mode to work in offline
# pyo.init_notebook_mode()
# or not...
# pyo.init_notebook_mode(connected=False) 



---



## Data File Specification

The following snippet takes a copy of the data file and attaches it to Colab's /content/ folder with the system `curl` command.    

Then the file is opened with Pandas `pd.csv_read()` to the gps_df dataframe.

In [ ]:
! curl 'https://raw.githubusercontent.com/rahulbhadani/medium.com/master/data/gps.csv' -o gps.csv
gps_data = "gps.csv"
gps_df = pd.read_csv(gps_data)

In [6]:
gps_df

,Time,Status,Long,Lat,Alt
0,1593297862,A,-82.423340,36.372966,485.2
1,1593297862,A,-82.423318,36.372949,485.3
2,1593297863,A,-82.423296,36.372931,485.3
3,1593297863,A,-82.423274,36.372914,485.3
4,1593297863,A,-82.423252,36.372898,485.4
...,...,...,...,...,...
949,1593297976,A,-82.399087,36.356296,478.8
950,1593297976,A,-82.399070,36.356281,478.6
951,1593297977,A,-82.399054,36.356265,478.5
952,1593297977,A,-82.399037,36.356250,478.4




---



## Downloading Plotly Charts as Images in Colab

*from: [PaulDeSalvo.com](https://www.pauldesalvo.com/how-to-download-plotly-express-charts-as-images-in-google-colab/)*

Before working with Mapbox and plotly, the following simple examples will be used to demonstrate downloading plotly charts as images in Google Colab.

### Interactive vs. Static Export

Plotly figures are interactive when viewed in a web browser: users can hover over data points, pan and zoom axes, and show and hide traces by clicking or double-clicking on the legend. Figures may be exported either to static image file formats like PNG, JPEG, SVG or PDF or exported to HTML files which can be opened in a browser and remain interactive. 

Additional Resource:   

[TutorialsPoint training on plotly](https://www.tutorialspoint.com/plotly/plotly_online_and_offline_plotting.htm)


**Interactive Export**

The `write_html()` plotly figure method is used to export an interactive image. 

The following snippet, which is commented-out, demonstrates the form of the function -- we will use this later with the gps.csv map data. 


In [ ]:
# fig.write_html('image.html')  # a practice exercise will use this later



---



**Static Export**

Uncomment and use `%%capture` as a magic command to hide the output instream in a Colab notebook.  The output image is created and stored to the object assigned but `%%capture` prevents Colab from displaying that same image.

In [ ]:
# Setup the required libraries and commands
# %%capture   # uncomment this statement to hide Colab image display
# !pip install kaleido  # already run at the start of this notebook 
# !pip install plotly>=4.0.0  # already run at the strt of this notebook
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
# import the graph_objects() method from plotly as go
import plotly.graph_objects as go

Once the snippet of code above is run, then a static chart of variable **fig** can be saved by using this code: `fig.write_image("image.png")`.    

***Here is an example of saving a Static Image:***

In [16]:
x = np.arange(10)
fig = go.Figure(data=go.Scatter(x=x, y=x**2))

In [ ]:
fig.show()

In [18]:
fig.write_image("image.png")



---



The following examples create and work with a simple scatter plot with 100 random points of varying color and size.  A random number generator is used to create the underlying data for the objects on the scatter plot.    

These examples are derived from the plotly [Graphic Libraries documentation online](https://plotly.com/python/static-image-export/). 


In [ ]:
# initiate a random number generator
np.random.seed(1)

# Establish an array of 100 x,y coordinate point with random color and size
N = 100
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N) * 30

# Draw an image of the data created
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode="markers",
    marker=go.scatter.Marker(
        size=sz,
        color=colors,
        opacity=0.6,
        colorscale="Viridis"
    )
))


In [ ]:
# Use show() to display the figure if it was supressed
fig.show()

***Writing Raster Graphic Formats***

Write the figure as a raster graphic in **PNG** file format 

In [20]:
fig.write_image("fig1.png")  # note: the string may include a qualified name

Write the figure as a raster graphic in **JPEG** file format 

In [21]:
fig.write_image("fig1.jpg")  # note: the string may include a qualified name

Write the figure as a raster graphic in **WebP** file format 

In [22]:
fig.write_image("fig1.webp")  # note: the string may include a qualified name

***Writing Vector Image Formats***

Write the figure as a vector image in **SVG** file format 

In [23]:
fig.write_image("fig1.svg")  # note: the string may include a qualified name

Write the figure as a vector image in **PDF** file format 

In [24]:
fig.write_image("fig1.pdf")  # note: the string may include a qualified name



---



## Mapbox and Plotly Figure Generation

Professor Patrick's public API token for Mapbox (don't go nuts with it, please!)    

`pk.eyJ1IjoicHJvZmVzc29ycGF0cmljayIsImEiOiJja3psbGY5eDcxZ3o5Mm9wYWNteXZmZ3A0In0.bwEt4H0OJv4PRHuZrlOQJg`

In [7]:
api_token = input("Enter public token for Mapbox: ")

Enter public token for Mapbox: pk.eyJ1IjoicHJvZmVzc29ycGF0cmljayIsImEiOiJja3psbGY5eDcxZ3o5Mm9wYWNteXZmZ3A0In0.bwEt4H0OJv4PRHuZrlOQJg




---



In [8]:
fig = px.scatter_mapbox(gps_df, lat="Lat", lon="Long", color='Time',
                  color_continuous_scale=["black", "purple", "red" ], size_max=30, zoom=12.5,
                  height = 600, width = 1000, #center = dict(lat = g.center)
                        title='Drive Route with Mapbox',
                       #mapbox_style="open-street-map"
                       )


fig.update_layout(font_size=16,  title={'xanchor': 'center','yanchor': 'top', 'y':0.9, 'x':0.5,}, 
        title_font_size = 24, mapbox_accesstoken=api_token, mapbox_style = "mapbox://styles/strym/ckhd00st61aum19noz9h8y8kw")
fig.update_traces(marker=dict(size=6))



---



## Mapbox Web Services    

The Mapbox web services APIs allow you to programmatically access Mapbox tools and services. You can use these APIs to retrieve information about your account, upload and change resources, use core Mapbox tools, and more.

Mapbox APIs are divided into four distinct services: **Maps, Navigation, Search, and Accounts**. Each of these services has its own overview page in the documentation. Overview pages are divided into the individual APIs that make up the service. The documentation for each API is structured by endpoints. An endpoint is a specific method within an API that performs one action and is located at a specific URL.    


1. [Maps service](https://docs.mapbox.com/api/maps/)
2. [Search service](https://docs.mapbox.com/api/search/)
3. [Navigation service](https://docs.mapbox.com/api/navigation/)
4. [Accounts service](https://docs.mapbox.com/api/accounts/) 

### Geocoding Mapbox API

One of the Mapbox search API endpoints is the Geocoding API which can be consider as an alternative to using a Google API service for looking up the longitude and latitude of a location.  Mapbox also provides functionality to lookup a longitude:latitude pair and return the nearby location and for snapping to a location. 

In [ ]:
# Endpoint for Mapbox geolocation API - replace {search_text} with a string 
https://api.mapbox.com/geocoding/v5/mapbox.places/{search_text}.json


Optional parameters for the Mapbox search geocode API include the following:

* proximity
* language
* country
* types - to specify tags of location types (place, postcode, etc.)
* limit - to restrict the number of items returned  



---



In [ ]:
# to display the current plotly figure object

fig.show()



---



## Kaleido Writing Figure to Files

[Kaleido](https://medium.com/plotly/introducing-kaleido-b03c4b7b1d81) is a cross-platform library for generating static images (e.g. png, svg, pdf, etc.) ... **js** images from Python for use by `plotly.py`, but it is designed to be relatively straight-forward to extend to other web-based visualization libraries, and other programming languages.    

The goal of the Kaleido project is to make static image export of web-based visualization libraries as universally available and reliable as it is in **matplotlib** and **ggplot2**.    


In [ ]:
# to display the plotly figure-object's methods and attributes

dir(fig)  

In [12]:
fig.write_image('gps.png')  # This particular write_...() seems finicky

In [ ]:
fig.write_html('gps.html')

**Plotly Image Export Settings (Kaleido)**

Various image export settings can be configured using the `plotly.io.kaleido`.scope object. For example, the `default_format` property can be used to specify that the default export format should be **svg** instead of **png**.

In [ ]:
import plotly.io as pio
pio.kaleido.scope.default_format = "svg"

Here is a complete listing of the available **Kaleido** image export settings:

* `default_width`: The default pixel width to use on image export.
* `default_height`: The default pixel height to use on image export.
* `default_scale`: The default image scale factor applied on image export.
* `default_format`: The default image format used on export. One of "png", "jpeg", "webp", "svg", "pdf", or "eps".
* `mathjax`: Location of the MathJax bundle needed to render LaTeX characters. Defaults to a CDN location. If fully offline export is required, set this to a local MathJax bundle.
* `topojson`: Location of the topojson files needed to render choropleth traces. Defaults to a CDN location. If fully offline export is required, set this to a local directory containing the Plotly.js topojson files.
* `mapbox_access_token`: The default Mapbox access token.



---



## Publishing on Chart Studio

from: [Getting Started with Chart Studio in plotly](https://plotly.com/python/getting-started-with-chart-studio/)    

Installation and Initialization Steps for Using Chart Studio in Python.

### Installing Chart Studio    

*(this step was completed at the start of this notebook -- repeating here for instructional purposes)*

To install Chart Studio's python package, use the package manager pip inside your terminal.    

If you don't have pip installed on your machine, [click here](https://pip.pypa.io/en/latest/installing.html) for pip's installation instructions.

`$ pip install chart_studio`
or
`$ sudo pip install chart_studio`

Plotly's Python package is installed alongside the Chart Studio package and it is updated frequently! To upgrade, run:

`$ pip install plotly --upgrade`

### Initialization for Online Plotting    


Chart Studio provides a web-service for hosting graphs. [Create a free account](https://plotly.com/api_signup) to get started. Graphs are saved inside a usre's online Chart Studio account.  Each user controls the privacy of their account. Public hosting is free, for private hosting, [see the paid plans](https://plotly.com/products/cloud/).

After installing the Chart Studio package, user credentials must be set in Python:

```
1. import chart_studio
2. chart_studio.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw')
```

Replace 'DemoAccount' and 'lr1c37zw81' with the Plotly username and API key.    

[Find user account API keys here](https://plotly.com/settings/api).

The initialization step places a special `.plotly/.credentials` file in the user's home directory. The `~/.plotly/.credentials` file should look something like this:    

```
{
"username": "DemoAccount",
"stream_ids": ["ylosqsyet5", "h2ct8btk1s", "oxz4fm883b"],
"api_key": "lr1c37zw81"
}
```



Alternatively, the next snippet shows how to take user input of the user name and api token to publish to that Plotly Chart Studio account online.     

Professor Patrick's credentials on plotly Chart Studio:

**User** - `professorpatrick` 

**Chart Studio API token** - `oBPoJkrGEAXCd4pqSAMO`

In [10]:
# Preparing requisite Chart Studio details
import chart_studio.plotly as py
# import chart_studio  # already imported at the start of this notebook
plotly_api_ket = input("Enter Chart Studio API Token: ")
user_name = input("Enter username: ")

Enter Chart Studio API Token: oBPoJkrGEAXCd4pqSAMO
Enter username: professorpatrick


In [11]:
# Setting credential and publishing to Chart Studio
chart_studio.tools.set_credentials_file(username=user_name, api_key=plotly_api_ket)
py.plot(fig, filename = 'mapbox_express', auto_open=True)

'https://plotly.com/~professorpatrick/1/'

An example output from MapBox to plotly Chart Studio can be found at:    

`https://plotly.com/~professorpatrick/1/`



---

